In [2]:
import os
import shutil
import yaml
from glob import glob
from ultralytics import YOLO
from roboflow import Roboflow

# --- 1. 기본 경로 및 최종 설정 ---
# --------------------------------------------------------------------------
# (사용자 확인 필요)
# --------------------------------------------------------------------------
# 모든 데이터셋과 결과가 저장될 상위 폴더 경로
BASE_DATA_PATH = 'C:/Users/Admin/work space/2nd/'

# Roboflow API 키
ROBOFLOW_API_KEY = "NjIXpou4o4gsuGClT8hI"

# 추가 학습을 시작할 모델 경로 (현재까지 가장 성능이 좋았던 modeling2의 결과)
PREVIOUS_MODEL_PATH = os.path.join(BASE_DATA_PATH, 'acne_model_8_classes_retrained', 'weights', 'best.pt')
# --------------------------------------------------------------------------

# --- 모든 데이터셋 정보 ---
# 기존 데이터셋
DATASET1_NAME = 'acne04-1'
DATASET2_NAME = 'Acne-detection-1'
# 추가 데이터셋 1
ROBOFLOW_WORKSPACE_3 = "acne-dqmyr"
ROBOFLOW_PROJECT_3 = "acne-detection-hjgkp"
ROBOFLOW_VERSION_3 = 9
DATASET3_NAME = f"{ROBOFLOW_PROJECT_3}-{ROBOFLOW_VERSION_3}"
# 추가 데이터셋 2
ROBOFLOW_WORKSPACE_4 = "test-l2t0m"
ROBOFLOW_PROJECT_4 = "acne-detection-nv9i6-ddxoy"
ROBOFLOW_VERSION_4 = 1
DATASET4_NAME = f"{ROBOFLOW_PROJECT_4}-{ROBOFLOW_VERSION_4}"

# --- 최종 결과 폴더 이름 설정 ---
COMBINED_DATASET_NAME = 'acne_dataset_final_ALL'
TRAINING_NAME = 'acne_model_final_ALL'

In [3]:
# --- 2. 새로운 Roboflow 데이터셋 다운로드 (필요시) ---
print("🚀 Roboflow 데이터셋 다운로드를 시작합니다...")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
# 데이터셋 3 다운로드
if not os.path.exists(os.path.join(BASE_DATA_PATH, DATASET3_NAME)):
    print(f"\n[1/2] '{DATASET3_NAME}' 데이터셋을 다운로드합니다...")
    project3 = rf.workspace(ROBOFLOW_WORKSPACE_3).project(ROBOFLOW_PROJECT_3)
    version3 = project3.version(ROBOFLOW_VERSION_3)
    version3.download("yolov11", location=BASE_DATA_PATH)
    print("✅ 다운로드 완료!")
else:
    print(f"\n[1/2] '{DATASET3_NAME}' 데이터셋이 이미 존재하여 다운로드를 건너뜁니다.")

# 데이터셋 4 다운로드
if not os.path.exists(os.path.join(BASE_DATA_PATH, DATASET4_NAME)):
    print(f"\n[2/2] '{DATASET4_NAME}' 데이터셋을 다운로드합니다...")
    project4 = rf.workspace(ROBOFLOW_WORKSPACE_4).project(ROBOFLOW_PROJECT_4)
    version4 = project4.version(ROBOFLOW_VERSION_4)
    version4.download("yolov11", location=BASE_DATA_PATH)
    print("✅ 다운로드 완료!")
else:
    print(f"\n[2/2] '{DATASET4_NAME}' 데이터셋이 이미 존재하여 다운로드를 건너뜁니다.")
print("-" * 50)


🚀 Roboflow 데이터셋 다운로드를 시작합니다...

[1/2] 'acne-detection-hjgkp-9' 데이터셋을 다운로드합니다...
loading Roboflow workspace...
loading Roboflow project...
✅ 다운로드 완료!

[2/2] 'acne-detection-nv9i6-ddxoy-1' 데이터셋을 다운로드합니다...
loading Roboflow workspace...
loading Roboflow project...
✅ 다운로드 완료!
--------------------------------------------------


In [4]:
# --- 3. 클래스 목록 정의 및 중복 클래스 확인 (수정된 경로 사용) ---
FINAL_CLASS_NAMES = sorted(['blackheads', 'comedone', 'cyst', 'fore', 'nodule', 'papule', 'pustule', 'whiteheads'])
final_class_set = set(FINAL_CLASS_NAMES)

def get_overlapping_classes(dataset_path, dataset_name):
    yaml_path = os.path.join(dataset_path, 'data.yaml')
    class_names = []
    if os.path.exists(yaml_path):
        with open(yaml_path, 'r', encoding='utf-8') as f:
            yaml_data = yaml.safe_load(f)
            class_names = yaml_data.get('names', [])
    
    overlapping = sorted(list(final_class_set.intersection(set(class_names))))
    
    print(f"\n--- {dataset_name} 클래스 정보 ---")
    print(f"전체 클래스: {class_names}")
    print(f"✅ 중복 클래스: {overlapping}")
    return class_names, overlapping

# ❗️❗️❗️[버그 수정]❗️❗️❗️ Roboflow 다운로드 폴더 경로를 직접 구성하여 정확하게 지정
src_path_d3_corrected = os.path.join(BASE_DATA_PATH, DATASET3_NAME)
src_path_d4_corrected = os.path.join(BASE_DATA_PATH, DATASET4_NAME)

d3_names, overlapping_d3 = get_overlapping_classes(src_path_d3_corrected, "데이터셋 3")
d4_names, overlapping_d4 = get_overlapping_classes(src_path_d4_corrected, "데이터셋 4")
print("-" * 50)



--- 데이터셋 3 클래스 정보 ---
전체 클래스: []
✅ 중복 클래스: []

--- 데이터셋 4 클래스 정보 ---
전체 클래스: []
✅ 중복 클래스: []
--------------------------------------------------


In [5]:
# --- 4. 클래스 ID 재매핑 테이블 생성 ---
final_name_to_id = {name: i for i, name in enumerate(FINAL_CLASS_NAMES)}
# 기존 데이터셋 매핑
ORIGINAL_D1_NAMES = ['fore', 'papule', 'pustule', 'nodule', 'whiteheads', 'cyst', 'blackheads']
remap_d1 = {i: final_name_to_id[name] for i, name in enumerate(ORIGINAL_D1_NAMES)}
ORIGINAL_D2_NAMES = ['comedone', 'pustule', 'papule', 'cyst', 'nodule']
remap_d2 = {i: final_name_to_id[name] for i, name in enumerate(ORIGINAL_D2_NAMES)}
# 추가 데이터셋 매핑
id_to_name_d3 = {i: name for i, name in enumerate(d3_names)}
remap_d3 = {i: final_name_to_id[name] for i, name in enumerate(d3_names) if name in overlapping_d3}
id_to_name_d4 = {i: name for i, name in enumerate(d4_names)}
remap_d4 = {i: final_name_to_id[name] for i, name in enumerate(d4_names) if name in overlapping_d4}


In [6]:
# --- 5. 데이터 병합 및 라벨 재매핑 실행 ---
src_path_d1 = os.path.join(BASE_DATA_PATH, DATASET1_NAME)
src_path_d2 = os.path.join(BASE_DATA_PATH, DATASET2_NAME)
combined_path = os.path.join(BASE_DATA_PATH, COMBINED_DATASET_NAME)

if os.path.exists(combined_path):
    shutil.rmtree(combined_path)
print(f"'{combined_path}' 폴더를 생성하고 4개 데이터셋 병합을 시작합니다...")

def process_dataset(src_path, remap_dict, dest_path, classes_to_keep=None, id_to_name_map=None):
    for split in ['train', 'valid', 'test']:
        src_image_dir = os.path.join(src_path, split, 'images')
        src_label_dir = os.path.join(src_path, split, 'labels')
        if not os.path.exists(src_image_dir): continue

        dest_image_dir = os.path.join(dest_path, split, 'images')
        dest_label_dir = os.path.join(dest_path, split, 'labels')
        os.makedirs(dest_image_dir, exist_ok=True)
        os.makedirs(dest_label_dir, exist_ok=True)

        for label_filename in glob(os.path.join(src_label_dir, '*.txt')):
            # ... (이하 함수 내용은 동일)
            base_filename = os.path.basename(label_filename)
            with open(label_filename, 'r') as f_in: lines = f_in.readlines()
            new_lines = []
            for line in lines:
                parts = line.strip().split();
                if not parts: continue
                original_id = int(parts[0])
                if classes_to_keep and id_to_name_map:
                    if id_to_name_map.get(original_id) not in classes_to_keep: continue
                if original_id in remap_dict:
                    parts[0] = str(remap_dict[original_id])
                    new_lines.append(' '.join(parts) + '\n')
            if new_lines:
                with open(os.path.join(dest_label_dir, base_filename), 'w') as f_out: f_out.writelines(new_lines)
                img_name_base = os.path.splitext(base_filename)[0]
                for ext in ['.jpg', '.jpeg', '.png']:
                    src_image_path = os.path.join(src_image_dir, img_name_base + ext)
                    if os.path.exists(src_image_path):
                        shutil.copy(src_image_path, os.path.join(dest_image_dir, img_name_base + ext)); break

print("1/4 - 기존 데이터셋 1 처리 중..."); process_dataset(src_path_d1, remap_d1, combined_path)
print("2/4 - 기존 데이터셋 2 처리 중..."); process_dataset(src_path_d2, remap_d2, combined_path)
print("3/4 - 추가 데이터셋 3 처리 중..."); process_dataset(src_path_d3_corrected, remap_d3, combined_path, classes_to_keep=overlapping_d3, id_to_name_map=id_to_name_d3)
print("4/4 - 추가 데이터셋 4 처리 중..."); process_dataset(src_path_d4_corrected, remap_d4, combined_path, classes_to_keep=overlapping_d4, id_to_name_map=id_to_name_d4)
print("✅ 데이터 병합 완료!"); print("-" * 50)

'C:/Users/Admin/work space/2nd/acne_dataset_final_ALL' 폴더를 생성하고 4개 데이터셋 병합을 시작합니다...
1/4 - 기존 데이터셋 1 처리 중...
2/4 - 기존 데이터셋 2 처리 중...
3/4 - 추가 데이터셋 3 처리 중...
4/4 - 추가 데이터셋 4 처리 중...
✅ 데이터 병합 완료!
--------------------------------------------------


In [7]:
# --- 6. 최종 data.yaml 파일 생성 ---
final_yaml_path = os.path.join(combined_path, 'data.yaml')
yaml_data = {'train': 'train/images', 'val': 'valid/images', 'test': 'test/images', 'nc': len(FINAL_CLASS_NAMES), 'names': FINAL_CLASS_NAMES}
with open(final_yaml_path, 'w', encoding='utf-8') as f:
    yaml.dump(yaml_data, f, allow_unicode=True, sort_keys=False)
print(f"✅ 최종 '{final_yaml_path}' 파일 생성 완료!"); print("-" * 50)

✅ 최종 'C:/Users/Admin/work space/2nd/acne_dataset_final_ALL\data.yaml' 파일 생성 완료!
--------------------------------------------------


In [10]:
# --- 7. 최종 모델 추가 학습 (Fine-tuning) ---
if not os.path.exists(PREVIOUS_MODEL_PATH):
     print(f"🚨 에러: 이전 모델 '{PREVIOUS_MODEL_PATH}'를 찾을 수 없습니다.")
else:
    print(f"🚀 이전 최강 모델 '{PREVIOUS_MODEL_PATH}'을 불러와 최종 학습을 시작합니다...")
    model = YOLO(PREVIOUS_MODEL_PATH)
    results = model.train(
        data=final_yaml_path,
        epochs=250,      # 더 많은 데이터로 학습하므로 epochs 증가 고려
        imgsz=640,
        batch=16,
        patience=50,
        project=BASE_DATA_PATH,
        name=TRAINING_NAME,
        workers=4,
        cache=True       # 데이터셋이 크므로 캐시 사용 권장
    )
    print(f"\n🎉 최종 학습 완료! 결과 저장 경로: {os.path.join(BASE_DATA_PATH, TRAINING_NAME)}")


🚀 이전 최강 모델 'C:/Users/Admin/work space/2nd/acne_model_8_classes_retrained\weights\best.pt'을 불러와 최종 학습을 시작합니다...
New https://pypi.org/project/ultralytics/8.3.202 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.200  Python-3.13.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/Admin/work space/2nd/acne_dataset_final_ALL\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01,

In [11]:
   # --- 8. 최종 모델 성능 검증 ---
print("\n🚀 테스트 데이터셋으로 최종 모델 성능을 검증합니다...")
best_model_path = os.path.join(BASE_DATA_PATH, TRAINING_NAME, 'weights', 'best.pt')
if os.path.exists(best_model_path):
        model = YOLO(best_model_path)
        metrics = model.val(data=final_yaml_path, split='test', imgsz=640)
        print("\n--- 최종 성능 지표 ---")
        print(f"mAP50-95: {metrics.box.map:.4f}")
        print(f"mAP50: {metrics.box.map50:.4f}")
        print(f"mAP75: {metrics.box.map75:.4f}")


🚀 테스트 데이터셋으로 최종 모델 성능을 검증합니다...
Ultralytics 8.3.200  Python-3.13.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 576.2894.6 MB/s, size: 200.2 KB)
val: Scanning C:\Users\Admin\work space\2nd\acne_dataset_final_ALL\test\labels.cache... 88 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 88/88 149.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 0.6it/s 10.6s0.4s
                   all         88        461      0.529      0.357      0.377      0.162
              comedone         24         86      0.702      0.466      0.518      0.204
                  cyst         18         19      0.675      0.549      0.574      0.294
                nodule         19         19      0.615      0.526      0.557      0.253
                papule         18        264     